In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data as data

from torchvision import transforms, datasets
from torcheval.metrics.functional import multiclass_f1_score

from tqdm.auto import tqdm


In [4]:
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
CLASS_NUM = 4

In [5]:
TEST_DATA_PATH = "datasets/real_data/test/"

# transform 적용
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

initial_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

test_data = datasets.ImageFolder(root=TEST_DATA_PATH, transform=initial_transform)
test_data_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# 불러온 data class 순서 확인
print(test_data.class_to_idx)
# 꽃개오동0 참취1 곰취2 개오동3
print(len(test_data)) # 698개 떠야함

{'Rhcrodhehd': 0, 'ckacnl': 1, 'rhacnl': 2, 'rodhehd': 3}
698


In [7]:
# model 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### model 불러오기
model = torch.load('saved_model/epoch_4_val_0.94.pt') ## 불러올 model 지정
model = model.to(device)

print(model)

loss_function = torch.nn.CrossEntropyLoss()

cuda
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride

In [25]:
import pdb
import numpy as np
from sklearn.metrics import classification_report

def valid(model, validloader, batch_size, loss_function):
    model.eval()
    valid_loss = 0
    correct = 0

    with torch.no_grad():
        # 모든 class f1-score 계산용
        all_pre = np.array([])
        all_lab = np.array([])
        # 각 class별 f1-score 계산용
        all_Rhc = []
        all_cka = []
        all_rha = []
        all_ro = []
        # 각 class별 accuracy 계산용
        all_correct = [0, 0, 0, 0]
        all_correct_num = [0, 0, 0, 0]
        
        for i, (image, label) in enumerate(tqdm(validloader)):
            image = image.to(device)
            label = label.to(device)
            output = model(image)
            output = model.CF_fc1(output)
            output = model.CF_fc2(output)
            # output = model.module.CF_fc(output) # DP사용할 때
            
            valid_loss += loss_function(output, label).item()    
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            # class별 accuracy 계산
            my_pred = prediction.squeeze()
            for idx, tm in enumerate(label):
                if tm == my_pred[idx]:
                    all_correct[tm] += 1
                all_correct_num[tm] += 1
                
            # f1 score 계산 위한 결과저장
            tmp = prediction.squeeze(-1).cpu().numpy()
            tmp2 = label.cpu().numpy()
            # class별 f1-score 계산

                
            # 모든 class의 f1-score 계산
            if all_pre.size == 0:
                all_pre = tmp
            else:
                all_pre = np.concatenate((all_pre,tmp))
            if all_lab.size == 0:
                all_lab = tmp2
            else:
                all_lab = np.concatenate((all_lab,tmp2))
                
        print(classification_report(all_lab, all_pre, digits=4))
        # f1-score 계산 위한 class별 label 생성
        # all_lab_Rhc = np.zeros_like(all_Rhc)
        # all_lab_cka = np.zeros_like(all_cka) + 1
        # all_lab_rha = np.zeros_like(all_rha) + 2
        # all_lab_ro = np.zeros_like(all_ro) + 3
    
    # f1_score 계산해서 저장
    # f1_score = []
    # f1_score.append(f1_score(torch.from_numpy(np.array(all_Rhc)), torch.from_numpy(all_lab_Rhc), num_classes=1))
    # f1_score.append(multiclass_f1_score(torch.from_numpy(np.array(all_cka)), torch.from_numpy(all_lab_cka), num_classes=1))
    # f1_score.append(multiclass_f1_score(torch.from_numpy(np.array(all_rha)), torch.from_numpy(all_lab_rha), num_classes=1))
    # f1_score.append(multiclass_f1_score(torch.from_numpy(np.array(all_ro)), torch.from_numpy(all_lab_ro), num_classes=1))
    # f1_score.append(multiclass_f1_score(torch.from_numpy(all_pre), torch.from_numpy(all_lab), num_classes=4))
    
    # valid_loss /= (len(validloader.dataset) / batch_size)
    # valid_accuracy = 100. * correct / len(validloader.dataset)
    for j in range(4):
        print("Class {0} accuracy : {1}".format(j, (len(np.where((all_pre != j) & (all_lab != j))[0]) + \
            len(np.where((all_pre == j) & (all_lab == j))[0])) / len(all_lab)))
    
    return valid_loss, all_pre, all_lab#, valid_accuracy, f1_score, all_correct, all_correct_num

In [26]:
test_loss, all_pre, all_lab = valid(model, test_data_loader, BATCH_SIZE, loss_function)


for j in range(4):
    print("Class {0} accuracy : {1}".format(j, (len(np.where((all_pre != j) & (all_lab != j))[0]) + \
        len(np.where((all_pre == j) & (all_lab == j))[0])) / len(all_lab)))
        
print() # 145, 540 / 147, 551 0.9813
        # 232, 458 / 234, 464 0.9885(len(np.where((all_pre != 2) & (all_lab != 2))[0]) + len(np.where((all_pre == 2) & (all_lab == 2))[0])) / len(all_lab)
        # 0.9813 , 0.9885, 0.9813, 0.9828
# print()
# print(f"test_accuracy: {test_accuracy} % f1-score: {f1_score}  ac: {ac}   acn: {acn}  class별 accuracy: {np.array(ac)/np.array(acn)}\n")

  0%|          | 0/44 [00:00<?, ?it/s]c:\Users\kser\Anaconda3\envs\sgmse\lib\site-packages\PIL\Image.py:2896: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 44/44 [02:08<00:00,  2.92s/it]

              precision    recall  f1-score   support

           0     0.9295    0.9864    0.9571       147
           1     0.9748    0.9915    0.9831       234
           2     0.9753    0.9461    0.9605       167
           3     0.9859    0.9333    0.9589       150

    accuracy                         0.9670       698
   macro avg     0.9664    0.9643    0.9649       698
weighted avg     0.9678    0.9670    0.9670       698

Class 0 accuracy : 0.9813753581661891
Class 1 accuracy : 0.9885386819484241
Class 2 accuracy : 0.9813753581661891
Class 3 accuracy : 0.9828080229226361
Class 0 accuracy : 0.9813753581661891
Class 1 accuracy : 0.9885386819484241
Class 2 accuracy : 0.9813753581661891
Class 3 accuracy : 0.9828080229226361



In [27]:
for j in range(4):
    print("Class {0} accuracy : {1}".format(j, (len(np.where((all_pre != j) & (all_lab != j))[0]) + \
        len(np.where((all_pre == j) & (all_lab == j))[0])) / len(all_lab)))

Class 0 accuracy : 0.9813753581661891
Class 1 accuracy : 0.9885386819484241
Class 2 accuracy : 0.9813753581661891
Class 3 accuracy : 0.9828080229226361


In [28]:
print(classification_report(all_lab, all_pre, digits=4))

              precision    recall  f1-score   support

           0     0.9295    0.9864    0.9571       147
           1     0.9748    0.9915    0.9831       234
           2     0.9753    0.9461    0.9605       167
           3     0.9859    0.9333    0.9589       150

    accuracy                         0.9670       698
   macro avg     0.9664    0.9643    0.9649       698
weighted avg     0.9678    0.9670    0.9670       698

